In [1]:
import os
import pickle
rna_list = []
folder = '/home/daniil/documents/projects/JH/rnpdiffusion/data/pkl/train'
train_pdb_lst = os.listdir(folder)
for idx, data in enumerate(train_pdb_lst):
    with open(os.path.join(folder, data, "RNA.pkl"), "rb") as f:
        RNA = pickle.load(f)
        for rna_key in RNA.keys():
            rna_list.append(RNA[rna_key]["RNA_seq_str"])


In [2]:
print(len(rna_list))

784


In [9]:
import torch
import torch.nn as nn
import fm

modelFM, alphabet = fm.pretrained.rna_fm_t12()
batch_converter = alphabet.get_batch_converter()

dataset = {}
for seq in rna_list:
    if len(seq) > 501 or len(seq) < 5:
        continue
    with torch.no_grad():
        _, _, batch_tokens = batch_converter([('seq',f"{seq}")])
        results = modelFM(batch_tokens, repr_layers=[12])
        token_embeddings = results["representations"][12]
        if seq not in dataset:
            dataset[seq] = token_embeddings

In [10]:
dataset

{'GGGCGGCUAGCUCAGCGGAAGAGCGCUCGCCUCACACGCGAGAGGUCGUAGGUUCAAGUCCUACGCCGCCCACCA': tensor([[[-0.2427, -0.0055,  0.0811,  ...,  0.0970, -0.2476, -0.0851],
          [-0.4073, -0.1451,  0.4022,  ..., -0.2470, -0.2225, -0.3328],
          [-0.3757, -0.1762,  0.3619,  ..., -0.4628,  0.0158, -0.5039],
          ...,
          [ 0.1211,  0.0619,  0.3032,  ..., -0.1187,  0.5033,  0.1129],
          [ 0.3015,  0.3730,  0.1141,  ..., -0.5503,  0.9108, -0.1103],
          [-0.1043,  0.5708, -0.2552,  ..., -0.3277,  0.1664, -0.2669]]]),
 'AAAAAAAAAAAAAAAAAAAAAAAAAAAA': tensor([[[-0.0364,  0.0762, -0.0982,  ...,  0.0477,  0.0076, -0.2534],
          [-0.0587, -0.0913,  0.3880,  ..., -0.2321,  0.0285, -0.1027],
          [-0.1874, -0.0965,  0.4268,  ..., -0.0404, -0.0545, -0.1025],
          ...,
          [ 0.0421,  0.0636,  0.2898,  ..., -0.2073,  0.2781, -0.1103],
          [-0.0698,  0.1726,  0.1477,  ..., -0.4042,  0.3399, -0.3005],
          [-0.2376,  0.1260, -0.2313,  ..., -0.4062,  0.5091, -0

In [11]:
seq = 'AGAAA'
dataset['AGAAA'].shape

torch.Size([1, 7, 640])

In [14]:
import torch
import torch.nn as nn

# Define the neural network class
class fnafmDecoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(fnafmDecoder, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

input_dim = 640  # Input dimension (last dimension of the input tensor)
output_dim = 4   # Output dimension (last dimension of the output tensor)

model = fnafmDecoder(input_dim, output_dim)

L = 10
input_tensor = torch.randn(1, L, input_dim)

# Pass the input tensor through the network to get the output tensor
output_tensor = model(input_tensor)

# Print the shapes to verify
print("Input shape:", input_tensor.shape)
print("Output shape:", output_tensor.shape)



Input shape: torch.Size([1, 10, 640])
Output shape: torch.Size([1, 10, 4])


In [22]:
dataset

{'GGGCGGCUAGCUCAGCGGAAGAGCGCUCGCCUCACACGCGAGAGGUCGUAGGUUCAAGUCCUACGCCGCCCACCA': tensor([[[-0.2427, -0.0055,  0.0811,  ...,  0.0970, -0.2476, -0.0851],
          [-0.4073, -0.1451,  0.4022,  ..., -0.2470, -0.2225, -0.3328],
          [-0.3757, -0.1762,  0.3619,  ..., -0.4628,  0.0158, -0.5039],
          ...,
          [ 0.1211,  0.0619,  0.3032,  ..., -0.1187,  0.5033,  0.1129],
          [ 0.3015,  0.3730,  0.1141,  ..., -0.5503,  0.9108, -0.1103],
          [-0.1043,  0.5708, -0.2552,  ..., -0.3277,  0.1664, -0.2669]]]),
 'AAAAAAAAAAAAAAAAAAAAAAAAAAAA': tensor([[[-0.0364,  0.0762, -0.0982,  ...,  0.0477,  0.0076, -0.2534],
          [-0.0587, -0.0913,  0.3880,  ..., -0.2321,  0.0285, -0.1027],
          [-0.1874, -0.0965,  0.4268,  ..., -0.0404, -0.0545, -0.1025],
          ...,
          [ 0.0421,  0.0636,  0.2898,  ..., -0.2073,  0.2781, -0.1103],
          [-0.0698,  0.1726,  0.1477,  ..., -0.4042,  0.3399, -0.3005],
          [-0.2376,  0.1260, -0.2313,  ..., -0.4062,  0.5091, -0

In [37]:
import numpy as np
import torch

def sequence_to_tensor(sequence):
    nucleotide_to_vector = {
        'A': [1, 0, 0, 0],
        'U': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'C': [0, 0, 0, 1]
    }
    
    one_hot_encoded = [nucleotide_to_vector[nucleotide] for nucleotide in sequence]  
    tensor = torch.tensor(one_hot_encoded, dtype=torch.float32).unsqueeze(0)
    
    return tensor


data = []
for key, value in dataset.items():
    data.append((value[:, 1:-1, :], sequence_to_tensor(key)))
data

[(tensor([[[-0.4073, -0.1451,  0.4022,  ..., -0.2470, -0.2225, -0.3328],
           [-0.3757, -0.1762,  0.3619,  ..., -0.4628,  0.0158, -0.5039],
           [-0.1933,  0.0349,  0.1978,  ..., -0.2336,  0.0489, -0.4097],
           ...,
           [-0.1117, -0.1924,  0.3244,  ...,  0.3401,  0.3024,  0.3153],
           [ 0.1211,  0.0619,  0.3032,  ..., -0.1187,  0.5033,  0.1129],
           [ 0.3015,  0.3730,  0.1141,  ..., -0.5503,  0.9108, -0.1103]]]),
  tensor([[[0., 0., 1., 0.],
           [0., 0., 1., 0.],
           [0., 0., 1., 0.],
           [0., 0., 0., 1.],
           [0., 0., 1., 0.],
           [0., 0., 1., 0.],
           [0., 0., 0., 1.],
           [0., 1., 0., 0.],
           [1., 0., 0., 0.],
           [0., 0., 1., 0.],
           [0., 0., 0., 1.],
           [0., 1., 0., 0.],
           [0., 0., 0., 1.],
           [1., 0., 0., 0.],
           [0., 0., 1., 0.],
           [0., 0., 0., 1.],
           [0., 0., 1., 0.],
           [0., 0., 1., 0.],
           [1., 0., 0

In [39]:
print(data[0][0].shape)
print(data[0][1].shape)
print(len(data))

torch.Size([1, 75, 640])
torch.Size([1, 75, 4])
431


In [40]:
def tensor_to_sequence(tensor):
    nucleotides = ['A', 'U', 'G', 'C']
    
    if tensor.dim() == 3:
        tensor = tensor.squeeze(0)
    
    sequence = ''.join([nucleotides[vector.argmax().item()] for vector in tensor])
    
    return sequence


In [48]:
def count_mismatches(true_seq, pred_seq):
    mismatches = 0
    for true_nuc, pred_nuc in zip(true_seq, pred_seq):
        if true_nuc != pred_nuc:
            mismatches += 1
    return mismatches

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_tensor, target_tensor = self.data[idx]
        return input_tensor, target_tensor

# Define the neural network class
class fnafmDecoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(fnafmDecoder, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

# Example data points for training (replace with your actual data)
train_data_points = data[:350]
val_data_points = data[:350]
# Create the datasets and dataloaders
train_dataset = CustomDataset(train_data_points)
val_dataset = CustomDataset(val_data_points)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Initialize the model, loss function, and optimizer
input_dim = 640
output_dim = 4
model = fnafmDecoder(input_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and evaluation loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_dataloader:
        # Flatten the batch of inputs and targets
        inputs = inputs.view(-1, input_dim)
        targets = targets.view(-1, output_dim)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

    # Evaluate on validation data
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_targets in val_dataloader:
            # Flatten the batch of validation inputs and targets
            val_inputs = val_inputs.view(-1, input_dim)
            val_targets = val_targets.view(-1, output_dim)

            # Forward pass
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_targets).item()
            # print('true:', tensor_to_sequence(val_targets))
            # print('pred:', tensor_to_sequence(val_outputs))
            print(f"{count_mismatches(tensor_to_sequence(val_targets), tensor_to_sequence(val_outputs))} out of {len(tensor_to_sequence(val_targets))}")

    # Print statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Loss: {running_loss / len(train_dataloader):.4f}, "
          f"Validation Loss: {val_loss / len(val_dataloader):.4f}")
    
torch.save(model.state_dict(), 'fnafmDecoder_weights.pth')
print("Training complete")


0 out of 75
0 out of 28
1 out of 62
0 out of 5
0 out of 18
0 out of 19
1 out of 27
1 out of 29
0 out of 77
0 out of 71
0 out of 75
0 out of 14
0 out of 92
1 out of 158
0 out of 25
0 out of 8
0 out of 8
0 out of 253
0 out of 20
0 out of 69
0 out of 46
0 out of 46
0 out of 17
0 out of 5
0 out of 9
0 out of 74
0 out of 17
0 out of 171
0 out of 97
0 out of 69
1 out of 157
0 out of 77
0 out of 8
0 out of 155
0 out of 5
0 out of 8
0 out of 76
0 out of 73
0 out of 9
0 out of 34
0 out of 44
0 out of 32
0 out of 74
0 out of 55
0 out of 7
0 out of 9
0 out of 20
0 out of 15
1 out of 83
0 out of 258
0 out of 6
0 out of 20
0 out of 18
0 out of 13
0 out of 13
0 out of 11
0 out of 72
0 out of 27
0 out of 41
0 out of 19
0 out of 32
0 out of 174
0 out of 77
0 out of 5
0 out of 74
1 out of 253
0 out of 67
0 out of 18
0 out of 9
0 out of 18
0 out of 14
0 out of 58
0 out of 5
1 out of 347
0 out of 7
0 out of 15
0 out of 233
0 out of 155
0 out of 9
0 out of 48
0 out of 79
0 out of 13
0 out of 33
0 out of 2

0.031759325154894213
